In [1]:
"""
Experiment 1: Model Comparison (Qwen 3B vs 7B)
Purpose: Compare latency, answer quality, and resource usage between models
Author: RAG Policy-as-Code Team
Date: December 2025
"""

import time
import pandas as pd
import numpy as np
import chromadb
import ollama
import json
import psutil
import os
from typing import List, Dict, Tuple
from datetime import datetime

# ============================================================================
# CONFIGURATION
# ============================================================================

CHROMA_DB_PATH = "../../Chroma_db_database"
COLLECTION_NAME = "policy_docs"
EMBEDDING_MODEL = "nomic-embed-text"
MODELS_TO_TEST = ["qwen2.5:3b-instruct", "qwen2.5:7b"]
TOP_K = 3  # Use optimal K from previous experiment

# Gold Standard Dataset (expanded for model comparison)
gold_standard = {
    "Q1": {
        "question": "What are the three main objectives of the SEDGs guidelines?",
        "gold_answer": "improving equitable access, extending basic facilities, setting up SEDGs Cell",
        "key_terms": ["equitable access", "facilities", "SEDGs Cell", "objectives"],
        "category": "List-type",
        "complexity": "Medium"
    },
    "Q2": {
        "question": "Who are included in Socio-Economically Disadvantaged Groups according to NEP 2020?",
        "gold_answer": "gender identity, social backwardness, educational backward, minority, disabilities, vulnerable groups, disadvantaged locations",
        "key_terms": ["SCs", "STs", "OBCs", "transgender", "minority", "disabilities", "EWS"],
        "category": "Complex-list",
        "complexity": "High"
    },
    "Q3": {
        "question": "What is the composition of the SEDGs Cell governance structure?",
        "gold_answer": "Chairperson, Senior Professor, In-charge of Internal Complaint Committee, Coordinator/Director of IQAC, SC/ST Representative, OBC Representative, Two Students Representatives, Assistant Registrar",
        "key_terms": ["Chairperson", "Professor", "IQAC", "SC/ST Representative", "OBC Representative", "Students", "Assistant Registrar"],
        "category": "Factual",
        "complexity": "Medium"
    },
    "Q5": {
        "question": "How many Ph.D. Excellence Citations does UGC award annually?",
        "gold_answer": "ten PhD Excellence Citations, two from each discipline",
        "key_terms": ["ten", "two", "citations", "disciplines", "annually"],
        "category": "Factual",
        "complexity": "Low"
    },
    "Q7": {
        "question": "What are the key criteria for shortlisting candidates for PhD Excellence Citation?",
        "gold_answer": "Originality, Innovation, Contribution to knowledge, Methodology, Clarity, Impact, References, Presentation, Defense",
        "key_terms": ["Originality", "Innovation", "Contribution", "Methodology", "Impact", "References"],
        "category": "Complex-list",
        "complexity": "High"
    },
    "Q9": {
        "question": "What are the three main objectives of NEP SAARTHI initiative?",
        "gold_answer": "create awareness, encourage participation, establish feedback mechanism",
        "key_terms": ["awareness", "participation", "feedback", "NEP 2020"],
        "category": "List-type",
        "complexity": "Medium"
    },
    "Q11": {
        "question": "What are the roles and responsibilities of NEP SAARTHI?",
        "gold_answer": "Work as ambassador, conduct awareness drives, organize activities, disseminate information, establish dialogue, prepare notes, attend programmes, collect feedback",
        "key_terms": ["ambassador", "awareness", "activities", "disseminate", "dialogue", "feedback"],
        "category": "Complex-list",
        "complexity": "High"
    },
    "Q13": {
        "question": "What are the four main objectives of University-Industry linkage guidelines?",
        "gold_answer": "promote R&D, develop skill sets, establish linkages, create training opportunities",
        "key_terms": ["R&D", "skill sets", "linkages", "training", "apprenticeship"],
        "category": "List-type",
        "complexity": "Medium"
    },
    "Q16": {
        "question": "What are the two categories of undergraduate internships?",
        "gold_answer": "enhancing employability and developing research aptitude",
        "key_terms": ["employability", "research aptitude", "categories"],
        "category": "Factual",
        "complexity": "Low"
    },
    "Q17": {
        "question": "How many credits are assigned for internship in UG degree programmes?",
        "gold_answer": "2-4 credits, 60-120 hours duration, 12 credits for research project",
        "key_terms": ["2-4 credits", "60-120 hours", "12 credits", "mandatory"],
        "category": "Factual",
        "complexity": "Low"
    },
    "Q18": {
        "question": "What are the 10 main objectives of engaging undergraduates in internship programmes?",
        "gold_answer": "Integration of workshop with workplace, Understanding world of work, Hybrid learning, Research aptitude, Emerging technologies, Entrepreneurial capabilities, Decision-making, Social imagery, Collaborative influence, Professional competency",
        "key_terms": ["workshop", "workplace", "hybrid", "research", "entrepreneurial", "decision-making", "collaborative", "professional"],
        "category": "Complex-list",
        "complexity": "High"
    },
    # Out-of-scope questions for hallucination testing
    "Q_OUT1": {
        "question": "What is the capital of France?",
        "gold_answer": "I don't have information about this",
        "key_terms": ["don't have information", "not available", "cannot answer"],
        "category": "Out-of-scope",
        "complexity": "N/A"
    },
    "Q_OUT2": {
        "question": "How do I apply for a student loan?",
        "gold_answer": "I don't have information about this",
        "key_terms": ["don't have information", "not available", "cannot answer"],
        "category": "Out-of-scope",
        "complexity": "N/A"
    },
}

# ============================================================================
# HELPER FUNCTIONS
# ============================================================================

def initialize_chromadb(db_path: str, collection_name: str):
    """Initialize ChromaDB client and get collection"""
    print(f"Initializing ChromaDB from {db_path}...")
    
    client = chromadb.PersistentClient(path=db_path)
    
    try:
        collection = client.get_collection(name=collection_name)
        doc_count = collection.count()
        print(f"✓ Collection '{collection_name}' loaded successfully")
        print(f"✓ Total documents in collection: {doc_count}")
        
    except Exception as e:
        print(f"✗ Error: Collection '{collection_name}' not found")
        print(f"✗ Details: {e}\n")
        raise ValueError(f"Collection '{collection_name}' does not exist.")
    
    return client, collection


def get_model_info(model: str) -> Dict:
    """Get model information from Ollama"""
    try:
        info = ollama.show(model)
        return {
            'model': model,
            'parameters': info.get('details', {}).get('parameter_size', 'Unknown'),
            'quantization': info.get('details', {}).get('quantization_level', 'Unknown'),
            'family': info.get('details', {}).get('family', 'Unknown')
        }
    except Exception as e:
        print(f"Warning: Could not get model info for {model}: {e}")
        return {'model': model, 'parameters': 'Unknown'}


def get_system_resources() -> Dict:
    """Get current system resource usage"""
    process = psutil.Process(os.getpid())
    return {
        'cpu_percent': psutil.cpu_percent(interval=0.1),
        'memory_mb': process.memory_info().rss / 1024 / 1024,
        'memory_percent': process.memory_percent()
    }


def embed_query(query: str, model: str = EMBEDDING_MODEL) -> List[float]:
    """Generate embedding for a query using Ollama"""
    try:
        response = ollama.embeddings(model=model, prompt=query)
        return response['embedding']
    except Exception as e:
        print(f"Error generating embedding: {e}")
        return None


def retrieve_segments(collection, query_embedding: List[float], top_k: int) -> Dict:
    """Retrieve top-k segments from ChromaDB"""
    try:
        results = collection.query(
            query_embeddings=[query_embedding],
            n_results=top_k,
            include=["documents", "metadatas", "distances"]
        )
        return results
    except Exception as e:
        print(f"Error retrieving segments: {e}")
        return None


def build_context(retrieved_results: Dict, top_k: int) -> str:
    """Build context string from retrieved results"""
    documents = retrieved_results['documents'][0][:top_k]
    metadatas = retrieved_results['metadatas'][0][:top_k]
    
    context_parts = []
    for i, (doc, meta) in enumerate(zip(documents, metadatas)):
        source = meta.get('source', f'Document_{i+1}')
        context_parts.append(f"[Source: {source}]\n{doc}\n")
    
    return "\n".join(context_parts)


def construct_rag_prompt(context: str, question: str) -> str:
    """Construct prompt for RAG"""
    prompt = f"""You are an academic compliance expert specializing in UGC guidelines and regulations.

INSTRUCTIONS:
1. Answer the question using ONLY the provided context below
2. If the question asks for multiple items (objectives, categories, etc.), list ALL of them
3. Be concise, accurate, and factual
4. If the context doesn't contain relevant information, clearly state: "I don't have information about this in the available guidelines"
5. Include source references when possible

Context:
{context}

Question: {question}

Answer:"""
    return prompt


def generate_answer_with_metrics(prompt: str, model: str) -> Dict:
    """Generate answer and collect performance metrics"""
    
    # Get baseline resources
    resources_before = get_system_resources()
    start_time = time.time()
    
    try:
        response = ollama.generate(
            model=model,
            prompt=prompt,
            options={
                'temperature': 0.3,
                'top_p': 0.9,
                'num_predict': 512,
                'num_ctx': 4096  # Context window
            }
        )
        
        # Calculate metrics
        end_time = time.time()
        latency = end_time - start_time
        resources_after = get_system_resources()
        
        # Extract tokens info if available
        total_duration = response.get('total_duration', 0) / 1e9  # Convert to seconds
        prompt_eval_duration = response.get('prompt_eval_duration', 0) / 1e9
        eval_duration = response.get('eval_duration', 0) / 1e9
        
        prompt_eval_count = response.get('prompt_eval_count', 0)
        eval_count = response.get('eval_count', 0)
        
        # Calculate tokens per second
        prompt_tokens_per_sec = prompt_eval_count / prompt_eval_duration if prompt_eval_duration > 0 else 0
        output_tokens_per_sec = eval_count / eval_duration if eval_duration > 0 else 0
        
        return {
            'answer': response['response'],
            'latency_sec': round(latency, 2),
            'total_duration_sec': round(total_duration, 2),
            'prompt_eval_duration_sec': round(prompt_eval_duration, 2),
            'eval_duration_sec': round(eval_duration, 2),
            'prompt_tokens': prompt_eval_count,
            'output_tokens': eval_count,
            'total_tokens': prompt_eval_count + eval_count,
            'prompt_tokens_per_sec': round(prompt_tokens_per_sec, 2),
            'output_tokens_per_sec': round(output_tokens_per_sec, 2),
            'cpu_usage_percent': round((resources_after['cpu_percent'] + resources_before['cpu_percent']) / 2, 2),
            'memory_mb': round(resources_after['memory_mb'], 2),
            'answer_length': len(response['response'])
        }
        
    except Exception as e:
        print(f"Error generating answer with {model}: {e}")
        return None


def evaluate_answer_quality(answer: str, gold_data: Dict) -> Dict:
    """Evaluate answer quality with multiple metrics"""
    
    answer_lower = answer.lower()
    key_terms = gold_data['key_terms']
    gold_answer = gold_data['gold_answer'].lower()
    category = gold_data['category']
    
    # Metric 1: Key term coverage
    terms_found = sum(1 for term in key_terms if term.lower() in answer_lower)
    key_term_coverage = terms_found / len(key_terms) if key_terms else 0
    
    # Metric 2: Completeness (for list-type questions)
    if category in ['List-type', 'Complex-list']:
        # Check if answer contains enumeration
        has_enumeration = any(marker in answer for marker in ['1.', '2.', '3.', '-', '•', '(i)', '(a)'])
        completeness_bonus = 0.1 if has_enumeration else 0
    else:
        completeness_bonus = 0
    
    # Metric 3: Hallucination detection (for out-of-scope)
    if category == 'Out-of-scope':
        refusal_terms = ["don't have information", "not available", "cannot answer", 
                        "not mentioned", "context doesn't contain", "no information"]
        has_refusal = any(term in answer_lower for term in refusal_terms)
        hallucination_score = 1.0 if has_refusal else 0.0
    else:
        hallucination_score = None
    
    # Metric 4: Answer relevance (basic check)
    relevance_terms = [term.lower() for term in key_terms[:3]]  # Check first 3 key terms
    relevance_score = sum(1 for term in relevance_terms if term in answer_lower) / len(relevance_terms) if relevance_terms else 0
    
    # Metric 5: Conciseness (penalize overly verbose answers)
    word_count = len(answer.split())
    conciseness_score = 1.0 if word_count < 200 else 0.8 if word_count < 300 else 0.6
    
    # Composite quality score
    if category == 'Out-of-scope':
        quality_score = hallucination_score  # Only check if it refuses correctly
    else:
        quality_score = (
            0.5 * key_term_coverage +
            0.3 * relevance_score +
            0.1 * conciseness_score +
            0.1 * completeness_bonus
        )
    
    return {
        'key_term_coverage': round(key_term_coverage, 3),
        'relevance_score': round(relevance_score, 3),
        'conciseness_score': round(conciseness_score, 3),
        'hallucination_score': hallucination_score,
        'quality_score': round(quality_score, 3),
        'terms_found': terms_found,
        'total_terms': len(key_terms),
        'word_count': word_count
    }


# ============================================================================
# MAIN EXPERIMENT FUNCTION
# ============================================================================

def run_model_comparison_experiment(
    collection,
    gold_standard: Dict,
    models: List[str],
    top_k: int = TOP_K
) -> pd.DataFrame:
    """
    Run model comparison experiment
    
    Args:
        collection: ChromaDB collection
        gold_standard: Dictionary with test questions
        models: List of model names to test
        top_k: Number of documents to retrieve
    
    Returns:
        DataFrame with results
    """
    
    results = []
    total_tests = len(gold_standard) * len(models)
    current_test = 0
    
    print(f"\n{'='*80}")
    print(f"STARTING MODEL COMPARISON EXPERIMENT")
    print(f"{'='*80}")
    print(f"Total tests to run: {total_tests}")
    print(f"Questions: {len(gold_standard)}")
    print(f"Models: {models}")
    print(f"Top-K: {top_k}\n")
    
    # Get model info
    print("Model Information:")
    print("-"*80)
    for model in models:
        info = get_model_info(model)
        print(f"  {model}: {info.get('parameters', 'Unknown')} parameters")
    print()
    
    for model in models:
        print(f"\n{'='*80}")
        print(f"Testing Model: {model}")
        print(f"{'='*80}")
        
        for q_id, gold_data in gold_standard.items():
            current_test += 1
            question = gold_data['question']
            category = gold_data['category']
            complexity = gold_data.get('complexity', 'Unknown')
            
            print(f"\n[{current_test}/{total_tests}] {q_id} ({category}, {complexity})")
            print(f"  Q: {question[:70]}...")
            
            try:
                # Step 1: Embed query (same for both models)
                query_embedding = embed_query(question)
                if query_embedding is None:
                    print("  ✗ Failed to generate embedding")
                    continue
                
                # Step 2: Retrieve segments
                retrieved_results = retrieve_segments(collection, query_embedding, top_k)
                if retrieved_results is None:
                    print("  ✗ Failed to retrieve segments")
                    continue
                
                # Step 3: Build context
                context = build_context(retrieved_results, top_k)
                context_length = len(context)
                
                # Step 4: Generate answer with metrics
                prompt = construct_rag_prompt(context, question)
                prompt_length = len(prompt)
                
                metrics = generate_answer_with_metrics(prompt, model)
                
                if metrics is None:
                    print("  ✗ Failed to generate answer")
                    continue
                
                # Step 5: Evaluate answer quality
                quality_metrics = evaluate_answer_quality(metrics['answer'], gold_data)
                
                # Print summary
                print(f"  ⏱  Latency: {metrics['latency_sec']:.2f}s | Tokens: {metrics['total_tokens']} | Speed: {metrics['output_tokens_per_sec']:.1f} tok/s")
                print(f"  📊 Quality: {quality_metrics['quality_score']:.3f} | Coverage: {quality_metrics['key_term_coverage']:.3f} | Words: {quality_metrics['word_count']}")
                
                # Store results
                result = {
                    'model': model,
                    'question_id': q_id,
                    'question': question,
                    'category': category,
                    'complexity': complexity,
                    'context_length': context_length,
                    'prompt_length': prompt_length,
                    **metrics,
                    **quality_metrics,
                    'answer_preview': metrics['answer'][:200] + "..." if len(metrics['answer']) > 200 else metrics['answer']
                }
                
                results.append(result)
                
            except Exception as e:
                print(f"  ✗ Error: {e}")
                continue
    
    print(f"\n{'='*80}")
    print(f"EXPERIMENT COMPLETED")
    print(f"{'='*80}\n")
    
    return pd.DataFrame(results)


def analyze_model_comparison(results_df: pd.DataFrame) -> Dict:
    """Analyze and compare models"""
    
    print("\n" + "="*80)
    print("ANALYSIS: MODEL COMPARISON RESULTS")
    print("="*80 + "\n")
    
    # Separate in-scope and out-of-scope
    in_scope_df = results_df[results_df['category'] != 'Out-of-scope']
    out_scope_df = results_df[results_df['category'] == 'Out-of-scope']
    
    print("1. OVERALL PERFORMANCE (In-Scope Questions):")
    print("-"*80)
    
    # Group by model
    summary = in_scope_df.groupby('model').agg({
        'latency_sec': ['mean', 'std', 'min', 'max'],
        'output_tokens_per_sec': ['mean', 'std'],
        'quality_score': ['mean', 'std'],
        'key_term_coverage': ['mean', 'std'],
        'word_count': 'mean',
        'total_tokens': 'mean',
        'cpu_usage_percent': 'mean',
        'memory_mb': 'mean'
    }).round(3)
    
    print(summary)
    print()
    
    # Compare models directly
    models = results_df['model'].unique()
    if len(models) == 2:
        model_3b = [m for m in models if '3b' in m.lower()][0]
        model_7b = [m for m in models if '7b' in m.lower()][0]
        
        df_3b = in_scope_df[in_scope_df['model'] == model_3b]
        df_7b = in_scope_df[in_scope_df['model'] == model_7b]
        
        print("2. HEAD-TO-HEAD COMPARISON:")
        print("-"*80)
        print(f"\n{'Metric':<30} {'3B Model':<15} {'7B Model':<15} {'Winner':<10}")
        print("-"*70)
        
        metrics_to_compare = [
            ('Avg Latency (s)', 'latency_sec', 'lower'),
            ('Avg Quality Score', 'quality_score', 'higher'),
            ('Avg Coverage', 'key_term_coverage', 'higher'),
            ('Tokens/sec', 'output_tokens_per_sec', 'higher'),
            ('Word Count', 'word_count', 'neither'),
            ('CPU Usage (%)', 'cpu_usage_percent', 'lower'),
            ('Memory (MB)', 'memory_mb', 'lower')
        ]
        
        comparison_results = {}
        
        for metric_name, metric_key, preference in metrics_to_compare:
            val_3b = df_3b[metric_key].mean()
            val_7b = df_7b[metric_key].mean()
            
            if preference == 'lower':
                winner = '3B ✓' if val_3b < val_7b else '7B ✓'
            elif preference == 'higher':
                winner = '3B ✓' if val_3b > val_7b else '7B ✓'
            else:
                winner = '-'
            
            print(f"{metric_name:<30} {val_3b:<15.3f} {val_7b:<15.3f} {winner:<10}")
            comparison_results[metric_key] = {'3b': val_3b, '7b': val_7b, 'winner': winner}
        
        print()
        
        # Performance by complexity
        print("3. PERFORMANCE BY COMPLEXITY:")
        print("-"*80)
        complexity_perf = in_scope_df.groupby(['model', 'complexity']).agg({
            'quality_score': 'mean',
            'latency_sec': 'mean'
        }).round(3)
        print(complexity_perf)
        print()
        
        # Performance by category
        print("4. PERFORMANCE BY QUESTION CATEGORY:")
        print("-"*80)
        category_perf = in_scope_df.groupby(['model', 'category']).agg({
            'quality_score': 'mean',
            'key_term_coverage': 'mean'
        }).round(3)
        print(category_perf)
        print()
        
        # Hallucination test results
        if len(out_scope_df) > 0:
            print("5. HALLUCINATION TEST (Out-of-Scope Questions):")
            print("-"*80)
            hallucination_results = out_scope_df.groupby('model').agg({
                'hallucination_score': ['mean', 'count']
            }).round(3)
            print(hallucination_results)
            print("\nNote: Score of 1.0 means model correctly refused to answer")
            print()
        
        return {
            'summary': summary,
            'comparison': comparison_results,
            'complexity_performance': complexity_perf,
            'category_performance': category_perf,
            'hallucination_test': hallucination_results if len(out_scope_df) > 0 else None
        }
    
    return {'summary': summary}


def save_results(results_df: pd.DataFrame, analysis: Dict, output_dir: str = "./experiment_results"):
    """Save results to files"""
    import os
    
    os.makedirs(output_dir, exist_ok=True)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Save detailed results
    csv_path = f"{output_dir}/model_comparison_{timestamp}.csv"
    results_df.to_csv(csv_path, index=False)
    print(f"✓ Detailed results saved to: {csv_path}")
    
    # Save summary
    summary_path = f"{output_dir}/model_summary_{timestamp}.csv"
    analysis['summary'].to_csv(summary_path)
    print(f"✓ Summary saved to: {summary_path}")
    
    # Save comparison as JSON
    json_path = f"{output_dir}/model_analysis_{timestamp}.json"
    analysis_serializable = {
        'comparison': analysis.get('comparison', {}),
        'timestamp': timestamp,
        'models_tested': MODELS_TO_TEST
    }
    with open(json_path, 'w') as f:
        json.dump(analysis_serializable, f, indent=2)
    print(f"✓ Analysis saved to: {json_path}\n")


def plot_model_comparison(results_df: pd.DataFrame, output_dir: str = "./experiment_results"):
    """Generate comparison visualizations"""
    try:
        import matplotlib.pyplot as plt
        import seaborn as sns
        
        sns.set_style("whitegrid")
        
        # Separate in-scope questions
        in_scope_df = results_df[results_df['category'] != 'Out-of-scope']
        
        fig, axes = plt.subplots(2, 3, figsize=(18, 10))
        fig.suptitle('Model Comparison: Qwen 3B vs 7B', fontsize=16, fontweight='bold')
        
        # Plot 1: Latency comparison
        in_scope_df.groupby('model')['latency_sec'].mean().plot(
            kind='bar', ax=axes[0, 0], color=['skyblue', 'salmon']
        )
        axes[0, 0].set_title('Average Latency', fontweight='bold')
        axes[0, 0].set_ylabel('Seconds')
        axes[0, 0].set_xlabel('')
        axes[0, 0].tick_params(axis='x', rotation=0)
        
        # Plot 2: Quality score
        in_scope_df.groupby('model')['quality_score'].mean().plot(
            kind='bar', ax=axes[0, 1], color=['skyblue', 'salmon']
        )
        axes[0, 1].set_title('Answer Quality Score', fontweight='bold')
        axes[0, 1].set_ylabel('Score')
        axes[0, 1].set_xlabel('')
        axes[0, 1].tick_params(axis='x', rotation=0)
        axes[0, 1].set_ylim([0, 1])
        
        # Plot 3: Tokens per second
        in_scope_df.groupby('model')['output_tokens_per_sec'].mean().plot(
            kind='bar', ax=axes[0, 2], color=['skyblue', 'salmon']
        )
        axes[0, 2].set_title('Generation Speed', fontweight='bold')
        axes[0, 2].set_ylabel('Tokens/sec')
        axes[0, 2].set_xlabel('')
        axes[0, 2].tick_params(axis='x', rotation=0)
        
        # Plot 4: Key term coverage
        in_scope_df.groupby('model')['key_term_coverage'].mean().plot(
            kind='bar', ax=axes[1, 0], color=['skyblue', 'salmon']
        )
        axes[1, 0].set_title('Key Term Coverage', fontweight='bold')
        axes[1, 0].set_ylabel('Coverage')
        axes[1, 0].set_xlabel('')
        axes[1, 0].tick_params(axis='x', rotation=0)
        axes[1, 0].set_ylim([0, 1])
        
        # Plot 5: Performance by complexity
        complexity_data = in_scope_df.groupby(['model', 'complexity'])['quality_score'].mean().unstack()
        complexity_data.plot(kind='bar', ax=axes[1, 1], width=0.7)
        axes[1, 1].set_title('Quality by Complexity', fontweight='bold')
        axes[1, 1].set_ylabel('Quality Score')
        axes[1, 1].set_xlabel('')
        axes[1, 1].tick_params(axis='x', rotation=0)
        axes[1, 1].legend(title='Complexity')
        
        # Plot 6: Resource usage
        resource_data = in_scope_df.groupby('model')[['cpu_usage_percent', 'memory_mb']].mean()
        resource_data['memory_gb'] = resource_data['memory_mb'] / 1024
        resource_data[['cpu_usage_percent']].plot(kind='bar', ax=axes[1, 2], color=['skyblue', 'salmon'])
        axes[1, 2].set_title('CPU Usage', fontweight='bold')
        axes[1, 2].set_ylabel('CPU %')
        axes[1, 2].set_xlabel('')
        axes[1, 2].tick_params(axis='x', rotation=0)
        
        plt.tight_layout()
        
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        plot_path = f"{output_dir}/model_comparison_plots_{timestamp}.png"
        plt.savefig(plot_path, dpi=300, bbox_inches='tight')
        print(f"✓ Plots saved to: {plot_path}")
        
        plt.close()
        
    except ImportError:
        print("⚠ matplotlib/seaborn not installed. Skipping plots.")


# ============================================================================
# MAIN EXECUTION
# ============================================================================

if __name__ == "__main__":
    
    print("\n" + "="*80)
    print("EXPERIMENT 1: MODEL COMPARISON (Qwen 3B vs 7B)")
    print("="*80 + "\n")
    
    # Initialize ChromaDB
    client, collection = initialize_chromadb(CHROMA_DB_PATH, COLLECTION_NAME)
    
    # Run experiment
    results_df = run_model_comparison_experiment(
        collection=collection,
        gold_standard=gold_standard,
        models=MODELS_TO_TEST,
        top_k=TOP_K
    )
    
    # Analyze results
    analysis = analyze_model_comparison(results_df)
    
    # Save results
    save_results(results_df, analysis)
    
    # Generate plots
    plot_model_comparison(results_df)
    
    print("\n" + "="*80)
    print("EXPERIMENT COMPLETED SUCCESSFULLY!")
    print("="*80 + "\n")
    
    # Display sample results
    print("Sample Results:")
    print(results_df[['model', 'question_id', 'latency_sec', 'quality_score', 'output_tokens_per_sec']].head(10))



EXPERIMENT 1: MODEL COMPARISON (Qwen 3B vs 7B)

Initializing ChromaDB from ../../Chroma_db_database...
✓ Collection 'policy_docs' loaded successfully
✓ Total documents in collection: 34

STARTING MODEL COMPARISON EXPERIMENT
Total tests to run: 26
Questions: 13
Models: ['qwen2.5:3b-instruct', 'qwen2.5:7b']
Top-K: 3

Model Information:
--------------------------------------------------------------------------------
  qwen2.5:3b-instruct: 3.1B parameters
  qwen2.5:7b: 7.6B parameters


Testing Model: qwen2.5:3b-instruct

[1/26] Q1 (List-type, Medium)
  Q: What are the three main objectives of the SEDGs guidelines?...
  ⏱  Latency: 14.12s | Tokens: 4189 | Speed: 47.8 tok/s
  📊 Quality: 0.235 | Coverage: 0.250 | Words: 77

[2/26] Q2 (Complex-list, High)
  Q: Who are included in Socio-Economically Disadvantaged Groups according ...
  ⏱  Latency: 5.83s | Tokens: 4232 | Speed: 43.6 tok/s
  📊 Quality: 0.110 | Coverage: 0.000 | Words: 106

[3/26] Q3 (Factual, Medium)
  Q: What is the compositio